In [28]:
from dotenv import load_dotenv
from openai import OpenAI # Version 1.33.0
from openai.types.beta.threads.message_create_params import Attachment, AttachmentToolFileSearch

load_dotenv()
client = OpenAI()   

In [29]:
import json

##### Carregando o arquivo do pedido

In [30]:
# Upload your pdf(s) to the OpenAI API
# file = client.files.create(
#     file=open('arquivos/pedido_iquine.pdf', 'rb'),
#     purpose='assistants'
# )

file = client.files.retrieve(file_id='file-uyoU2QBN26FGOhYqLjzW6xKM')

##### Definindo o assistant

In [31]:
# Create an Assistant (or fetch it if it was already created). It has to have
# "file_search" tool enabled to attach files when prompting it.
def get_assistant():
    for assistant in client.beta.assistants.list():
        if assistant.name == 'Assistente de compras':
            return assistant

    # No Assistant found, create a new one
    return client.beta.assistants.create(
        model='gpt-4o',
        description='Você é um assistente de compras de uma distribuidora de materiais para construção.',
        instructions="Você ajuda a extrair de um pedido de compra no formato pdf os itens contidos nele e retorna os dados em formato json.",
        tools=[{"type": "file_search"}],
        # response_format={"type": "json_object"}, # Isn't possible with "file_search"
        name='Assistente de compras',
    )

##### Criando a Thread das mensagens

In [32]:
# Create thread
thread = client.beta.threads.create()

##### Criando uma mensagem

In [33]:
# Add your prompt here
prompt = "leia todas as páginas do pdf e retorne todos os itens econtrados no formato json válido."
client.beta.threads.messages.create(
    thread_id = thread.id,
    role='user',
    content=prompt,
    attachments=[Attachment(file_id=file.id, tools=[AttachmentToolFileSearch(type='file_search')])]
)


Message(id='msg_RNQnMXJGroyr4Stz050h4Z5F', assistant_id=None, attachments=[Attachment(file_id='file-uyoU2QBN26FGOhYqLjzW6xKM', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='leia todas as páginas do pdf e retorne todos os itens econtrados no formato json válido.'), type='text')], created_at=1725651363, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_wvGrAao6zF5TWvGRMLtMCNa7')

##### Enviando a mensagem

In [34]:
# Run the created thread with the assistant. It will wait until the message is processed.
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=get_assistant().id,
    max_prompt_tokens=50000,
    timeout=300, # 5 minutes
    # response_format={"type": "json_object"}, # Isn't possible
)

##### Verificando se a mensagem foi processada corretamente

In [35]:
# Eg. issue with openai server
if run.status != "completed":
    raise Exception('Run failed:', run.status)

##### Pegando a resposta de retorno

In [36]:
# Fetch outputs of the thread
messages_cursor = client.beta.threads.messages.list(thread_id=thread.id)
messages = [message for message in messages_cursor]

In [39]:
run.usage

Usage(completion_tokens=2421, prompt_tokens=6599, total_tokens=9020)

In [38]:
message = messages[0] # This is the output from the Assistant (second message is your message)
assert message.content[0].type == "text"

# Output text of the Assistant
res_txt = message.content[0].text.value

if res_txt.startswith('```json'):
    res_txt = res_txt[6:]
if res_txt.endswith('```'):
    res_txt = res_txt[:-3]
res_txt = res_txt[:res_txt.rfind('}')+1]
res_txt = res_txt[res_txt.find('{'):]
res_txt.strip()

print(res_txt)

# data = json.loads(res_txt)
# print(data)

{
    "itens": [
        {"Referencia": "11.124", "Descrição": "TEE SOLD PVC DE 50X50mm", "Qtde": "150,000", "Preço Líq": "2,52"},
        {"Referencia": "12.901", "Descrição": "JOELHO AZUL C/BUCHA LATAO 90o 25X1/2\"", "Qtde": "600,000", "Preço Líq": "1,54"},
        {"Referencia": "11.268", "Descrição": "TEE PVC ESG SN DN 50", "Qtde": "200,000", "Preço Líq": "1,91"},
        {"Referencia": "11.270", "Descrição": "TEE PVC ESG SN DN 100", "Qtde": "100,000", "Preço Líq": "4,37"},
        {"Referencia": "11.238", "Descrição": "JOELHO PVC ESG SN 45o DN 50", "Qtde": "100,000", "Preço Líq": "1,18"},
        {"Referencia": "11.239", "Descrição": "JOELHO PVC ESG SN 45o DN 75", "Qtde": "120,000", "Preço Líq": "2,24"},
        {"Referencia": "11.240", "Descrição": "JOELHO PVC ESG SN 45o DN 100", "Qtde": "100,000", "Preço Líq": "3,38"},
        {"Referencia": "11.919", "Descrição": "JOELHO PVC ESG SN 90o DN 200*", "Qtde": "10,000", "Preço Líq": "50,39"},
        {"Referencia": "11.254", "Descriçã